In [233]:
# 1. Statically calculate. 

In [234]:
import z3

In [235]:
# First, encode the DNN
# (weights, bias, use activation function relu or not)
n00 = ([1.0, -1.0], 0.0, True)
n01 = ([1.0, 1.0], 0.0, True)
hidden_layer0 = [n00,n01]

n10 = ([0.5, -0.2], 0.0, True)
n11 = ([-0.5, 0.1], 0.0, True)
hidden_layer1 = [n10, n11]

# don't use relu for outputs
o0 = ([1.0, -1.0], 0.0, False)  
o1 = ([-1.0, 1.0], 0.0, False)
output_layer = [o0, o1]

dnn = [hidden_layer0, hidden_layer1, output_layer]

In [236]:
# define relu using z3
def relu(x):
    return z3.If(x > 0, x, 0)

In [237]:
# variables
i0, i1, n00, n01, n10, n11, o0, o1 = z3.Reals('i0 i1 n00 n01 n10 n11 o0 o1')
symbolic_states = []

# equation on n00
expr = n00 == relu(i0 - i1)
symbolic_states.append(expr)

# equation on n01
expr = n01 == relu(i0 + i1)
symbolic_states.append(expr)

# equation on n10
expr = n10 == relu( 0.5 * n00 - 0.2 * n01)
symbolic_states.append(expr)

# equation on n11
expr = n11 == relu(-0.5 * n00 + 0.1 * n01)
symbolic_states.append(expr)

# equation on o0
expr = o0 == n10 - n11
symbolic_states.append(expr)

# equation on o1
expr = o1 == -n10 + n11
symbolic_states.append(expr)

# conjunction
symbolic_states = z3.And(symbolic_states)

print(symbolic_states)

And(n00 == If(i0 - i1 > 0, i0 - i1, 0),
    n01 == If(i0 + i1 > 0, i0 + i1, 0),
    n10 == If(1/2*n00 - 1/5*n01 > 0, 1/2*n00 - 1/5*n01, 0),
    n11 ==
    If(-1/2*n00 + 1/10*n01 > 0, -1/2*n00 + 1/10*n01, 0),
    o0 == n10 - n11,
    o1 == -n10 + n11)


In [238]:
assert z3.is_expr(symbolic_states)

In [239]:
z3.solve(symbolic_states)

[n01 = 10,
 n00 = 2,
 i0 = 6,
 o1 = 0,
 n10 = 0,
 i1 = 4,
 o0 = 0,
 n11 = 0]


In [240]:
print("Prove that if (n0_0 > 0.0 and n0_1 <= 0.0) then o0 > o1")
g = z3.Implies(z3.And([n0_0 > 0.0, n0_1 <= 0.0]), o0 > o1)
print(g)  # Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
z3.prove(z3.Implies(symbolic_states, g))  # proved

Prove that if (n0_0 > 0.0 and n0_1 <= 0.0) then o0 > o1
Implies(And(n0_0 > 0, n0_1 <= 0), o0 > o1)
counterexample
[n01 = 15/2,
 n00 = 1/2,
 i0 = 4,
 n10 = 0,
 n11 = 1/2,
 n0_0 = 1/2,
 i1 = 7/2,
 o1 = 1/2,
 n0_1 = 0,
 o0 = -1/2]


In [241]:
print("Prove that when (i0 - i1 > 0 and i0 + i1 <= 0), then o0 > o1")
g = z3.Implies(z3.And([i0 - i1 > 0.0, i0 + i1 <= 0.0]), o0 > o1)
print(g)  # Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
z3.prove(z3.Implies(symbolic_states, g))
# proved

Prove that when (i0 - i1 > 0 and i0 + i1 <= 0), then o0 > o1
Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
proved


In [242]:
print("Disprove that when i0 - i1 >0, then o0 > o1")
g = z3.Implies(i0 - i1 > 0.0, o0 > o1)
print(g)  # Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
z3.prove(z3.Implies(symbolic_states, g))

Disprove that when i0 - i1 >0, then o0 > o1
Implies(i0 - i1 > 0, o0 > o1)
counterexample
[n01 = 10,
 n00 = 2,
 i1 = 4,
 i0 = 6,
 n11 = 0,
 o1 = 0,
 o0 = 0,
 n10 = 0]


In [243]:
# 2. Dynamically calculate.

In [ ]:
from collections import deque
import z3

def relu(x):
    return z3.If(x > 0, x, 0)

def my_symbolic_execution(dnn):
    symbolic_states = []

    i0, i1 = z3.Reals('i0 i1')
    queue = deque([(i0, i1)])

    for layer_idx, layer in enumerate(dnn):

        # neurons of previous layer
        previous_layer_values = queue.popleft()
        value_0 = previous_layer_values[0]
        value_1 = previous_layer_values[1]

        layer_exprs = []

        # TODO: handle negative numbers
        weight_0, bias_0, use_relu_0 = layer[0]
        weight_1, bias_1, use_relu_1 = layer[1]
        weight_0[0], weight_0[1], bias_0 = z3.RealVal(weight_0[0]), z3.RealVal(weight_0[1]), z3.RealVal(bias_0)
        weight_1[0], weight_1[1], bias_1 = z3.RealVal(weight_1[0]), z3.RealVal(weight_1[1]), z3.RealVal(bias_1)

        # Convert to z3 variables
        if layer_idx != len(dnn) - 1:
            neuron_0 = z3.Real(f'n{layer_idx}_0')
            neuron_1 = z3.Real(f'n{layer_idx}_1')

        else:
            o0, o1 = z3.Reals('o0 o1')
            neuron_0 = o0
            neuron_1 = o1

        # Use relur or not
        if use_relu_0:
            neuron_expr = neuron_0 == relu(z3.simplify((value_0 * weight_0[0] + value_1 * weight_0[1]) + bias_0))
            symbolic_states.append(neuron_expr)
            neuron_expr = neuron_1 == relu(z3.simplify((value_0 * weight_1[0] + value_1 * weight_1[1]) + bias_0))
            symbolic_states.append(neuron_expr)

        # append to the queue
        if layer_idx != len(dnn) - 1:
            next_layer_values = [(neuron_0, neuron_1)]
            queue.extend(next_layer_values)

    symbolic_states = z3.And(symbolic_states)
    print(symbolic_states)
    return symbolic_states

if __name__ == '__main__':
    # Example DNN
    dnn = [
        [([1.0, -1.0], 0.0, True), ([1.0, 1.0], 0.0, True)],
        [([0.5, -0.2], 0.0, True), ([-0.5, 0.1], 0.0, True)],
        [([1.0, -1.0], 0.0, False), ([-1.0, 1.0], 0.0, False)]
    ]

    symbolic_states = my_symbolic_execution(dnn)

And(n0_0 == If(i0 + -1*i1 > 0, i0 + -1*i1, 0),
    n0_1 == If(i0 + i1 > 0, i0 + i1, 0),
    n1_0 ==
    If(1/2*n0_0 + -1/5*n0_1 > 0, 1/2*n0_0 + -1/5*n0_1, 0),
    n1_1 ==
    If(-1/2*n0_0 + 1/10*n0_1 > 0, -1/2*n0_0 + 1/10*n0_1, 0))


In [245]:
print("Prove that if (n0_0 > 0.0 and n0_1 <= 0.0) then o0 > o1")
g = z3.Implies(z3.And([n0_0 > 0.0, n0_1 <= 0.0]), o0 > o1)
print(g)  # Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
z3.prove(z3.Implies(symbolic_states, g))  # proved

Prove that if (n0_0 > 0.0 and n0_1 <= 0.0) then o0 > o1
Implies(And(n0_0 > 0, n0_1 <= 0), o0 > o1)
counterexample
[i1 = -3/8,
 n1_0 = 1/4,
 n0_0 = 1/2,
 n1_1 = 0,
 i0 = 1/8,
 n0_1 = 0,
 o1 = 0,
 o0 = 0]


In [246]:
print("Prove that when (i0 - i1 > 0 and i0 + i1 <= 0), then o0 > o1")
g = z3.Implies(z3.And([i0 - i1 > 0.0, i0 + i1 <= 0.0]), o0 > o1)
print(g)  # Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
z3.prove(z3.Implies(symbolic_states, g))
# proved

Prove that when (i0 - i1 > 0 and i0 + i1 <= 0), then o0 > o1
Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
counterexample
[i1 = -1/4,
 n1_0 = 1/4,
 n1_1 = 0,
 i0 = 1/4,
 n0_1 = 0,
 o1 = 0,
 o0 = 0,
 n0_0 = 1/2]


In [247]:
print("Disprove that when i0 - i1 >0, then o0 > o1")
g = z3.Implies(i0 - i1 > 0.0, o0 > o1)
print(g)  # Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
z3.prove(z3.Implies(symbolic_states, g))

Disprove that when i0 - i1 >0, then o0 > o1
Implies(i0 - i1 > 0, o0 > o1)
counterexample
[n1_1 = 0,
 i1 = 1/2,
 i0 = 3/4,
 n0_1 = 5/4,
 o0 = 0,
 n1_0 = 0,
 o1 = 0,
 n0_0 = 1/4]
